In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [1]:
%%loadFromPOM
<dependency>
    <groupId>com.google.guava</groupId>
    <artifactId>guava</artifactId>
    <version>31.0.1-jre</version>
</dependency>
<dependency>
    <groupId>org.apache.logging.log4j</groupId>
    <artifactId>log4j-core</artifactId>
    <version>2.16.0</version>
</dependency>
<dependency>
  <groupId>org.apache.logging.log4j</groupId>
  <artifactId>log4j-api</artifactId>
  <version>2.16.0</version>
</dependency>

<dependency>
    <groupId>com.lmax</groupId>
    <artifactId>disruptor</artifactId>
    <version>3.4.4</version>
</dependency>

In [2]:
import com.google.common.util.concurrent.ListenableFuture;
import com.google.common.util.concurrent.ListeningExecutorService;
import com.google.common.util.concurrent.MoreExecutors;
import com.google.common.util.concurrent.Futures;
import com.google.common.util.concurrent.FluentFuture;
import com.google.common.util.concurrent.FutureCallback;
import com.google.common.util.concurrent.AsyncFunction;
import java.lang.Thread;
import java.util.function.Function;
import java.util.concurrent.TimeUnit;
import java.lang.Void;
import java.lang.Runnable;
import java.util.List;
import java.util.ArrayList;
import org.apache.logging.log4j.Logger;
import org.apache.logging.log4j.LogManager;
import org.apache.logging.log4j.Level;
import org.apache.logging.log4j.core.LoggerContext;
import org.apache.logging.log4j.core.config.Configurator;

In [3]:
Configurator.setRootLevel(Level.DEBUG);
Logger logger = LogManager.getLogger("AsyncJava");

# ListenableFuture\<T>
----------------

### ListenableFuture Execution Flow and Methods
* addListener
* cancel
* isCancelled
* isDone
* notify
* notifyAll
    
![ListenableFuture Execution Flow](static/LF.gif)

### Submitting a task

When submiting a long running task (implemented as a <a href=https://www.geeksforgeeks.org/difference-between-callable-and-runnable-in-java/>Callable</a>) to an executor to be ran on a seperate thread, the executor immediately returns a Future. If you want a ListenableFuture, you must decorate a normal executor object with the **listeningDecorator** method from the **MoreExecutors** library. This ListenableFuture<T> object is a placeholder for the future result that will be populated from the async running task.
    
    


In [4]:
// The executor is used to submit asynchrounous jobs. It manages the threadpool and tasks execution
final ListeningExecutorService executor = MoreExecutors.listeningDecorator(Executors.newFixedThreadPool(5));


// Submit Some Long Running Task To Be Executed Asynchronously
ListenableFuture<String> future = executor.submit(
   () -> {
         //Some Long running Task
         TimeUnit.MILLISECONDS.sleep(1000);
         logger.info("Executed Long Running Task On Thread {}\n", Thread.currentThread());
         return Thread.currentThread().getName();
        }
    
)

### ListenableFuture get()

This ListenableFuture<T> is used to get the result whenver it is available by calling ListenableFuture<T>.get(). If the async running tasks has finished by the time .get() is called, the result is return immediately. However if the result has not been produced yet by the async running task, the .get() method will be blocking on the main thread. You can check for the result without blocking by calling the ListenableFuture<T>.isDone() method.
    
    
 

In [5]:

while (!future.isDone()){
    logger.info("Async Task Is Still Running");
    TimeUnit.MILLISECONDS.sleep(300);
}

logger.info("The Async Task Was Running On Thread {}\nWe Are Running on Thread {}", 
                  future.get(),
                  Thread.currentThread());

16:12:11.066 [IJava-executor-0] INFO  AsyncJava - Async Task Is Still Running
16:12:11.378 [IJava-executor-0] INFO  AsyncJava - Async Task Is Still Running
16:12:11.678 [IJava-executor-0] INFO  AsyncJava - Async Task Is Still Running
16:12:11.976 [pool-2-thread-1] INFO  AsyncJava - Executed Long Running Task On Thread Thread[pool-2-thread-1,5,main]

16:12:12.007 [IJava-executor-0] INFO  AsyncJava - The Async Task Was Running On Thread pool-2-thread-1
We Are Running on Thread Thread[IJava-executor-0,5,main]


### Adding A Listener

What makes a ListenableFuture<T> a ListenableFuture is the ability to add a listener to it. The Listener will run when the future completes either succesfully or exceptionally. This registered Listener will run, unpredictablly, on some thread from the executor pool. Also note that you dont get access to the result of the listener. The listener must implement the Runnable interface which does not have a return value.
    
![ListenableFuture Execution Flow](static/LFL.gif)

In [6]:
// CREATE LF BY SUBMITTING A TASK TO THE EXECUTOR
ListenableFuture<String> futureToAddListener = executor.submit(
    () -> {
            //Some long running task
            TimeUnit.MILLISECONDS.sleep(3000);
            logger.info("Async Task Finished!\n");
            
            return "Hello";
        }
    );


// ADD LISTENER TO THE ListenableFuture
futureToAddListener.addListener(
    () -> {
       logger.info("Listener Executed on Thread {}\n NOTE: The Listener Won't Necessarily Run On The Same Thread As The Submitted Task", Thread.currentThread());
    }
 ,executor);


//Get Result From Long Running Task
logger.info("{} World!\n", futureToAddListener.get());


16:12:15.088 [pool-2-thread-2] INFO  AsyncJava - Async Task Finished!

16:12:15.088 [IJava-executor-0] INFO  AsyncJava - Hello World!

16:12:15.089 [pool-2-thread-3] INFO  AsyncJava - Listener Executed on Thread Thread[pool-2-thread-3,5,main]
 NOTE: The Listener Won't Necessarily Run On The Same Thread As The Submitted Task


# Chaining Async Tasks


## Async Steps to Bake Cupcakes

It is possible to chain Async tasks such that they execute whatever logic you like upon completion. This allows the user to not have to use the blocking call of the .get() method. 

For example let's say we want to do the following tasks in order to make cupcakes. We need to

1. Preheat the oven 
    * This task can run while we are doing everything else
2. Collect the ingredients
    * Eggs, Milk, & Butter are in the Fridge and can be grabbed together (synchronously)
    * Flour, Sugar, Baking Soda, Icing are in the Pantry and can be grabbed together (synchronously)
    * Foil cups and the cupcake pan are in the same cupboard and can be grabbed together (synchronously)
3. Mix the ingredients together (requires all ingredients first)
4. Fill the cupcake pan and place in the oven
5. Bake for 45 minutes


### DAG

![Bake a Cake](static/Async_Chaining.jpg)



From the Directed Acyclic Graph (DAG) we can see some steps are isolated from each other and can be ran asynchronously (Preheating oven and gathering items). But there are some steps that have dependencies on on prior running async tasks.

#### Step 1 - Preheat the oven


It will take a long time for the oven to preheat. So we can start it and then work on all our other tasks

<img src="static/preheatOven.png" height="20%" width="20%">

In [7]:
//Preheat the oven
ListenableFuture<Boolean> ovenPreheated = executor.submit(
    () -> { 
            // The oven will take awhile to heat up
            logger.info("Preheating Oven");
            Thread.sleep(10000);
            return true;
    }
);

#### Step 2 - Collect the ingredients
Each step of collecting items from their respective locations will be their own Async Task / ListenableFuture

<img src="static/gatherItems.png" height="10%" width="10%">

In [8]:
// Items from the fridge
ListenableFuture<List<String>> fridgeItems = executor.submit(
    () -> {
        ArrayList<String> items = new ArrayList<String>();

        logger.info("Getting items from the fridge");
        TimeUnit.MILLISECONDS.sleep(1000);
        items.add("Eggs");
        TimeUnit.MILLISECONDS.sleep(1000);
        items.add("Milk");
        TimeUnit.MILLISECONDS.sleep(1000);
        items.add("Butter");
        TimeUnit.MILLISECONDS.sleep(1000);
        
        return items;
    }
);
    
    
// Items from the pantry
ListenableFuture<List<String>> pantryItems = executor.submit(
    () -> {
        ArrayList<String> items = new ArrayList<String>();

        logger.info("Getting items from the pantry");
        TimeUnit.MILLISECONDS.sleep(1000);
        items.add("Flour");
        TimeUnit.MILLISECONDS.sleep(1000);
        items.add("Sugar");
        TimeUnit.MILLISECONDS.sleep(1000);
        items.add("Baking Soda");
        TimeUnit.MILLISECONDS.sleep(1000);
        
        return items;
    }
);
    
    
// Items from the cupboard
ListenableFuture<List<String>> cupboardItems = executor.submit(
    () -> {
        ArrayList<String> items = new ArrayList<String>();

        logger.info("Getting items from the cupboard");
        TimeUnit.MILLISECONDS.sleep(1000);
        items.add("Foil Cups");
        TimeUnit.MILLISECONDS.sleep(1000);
        items.add("Baking Pan");
        TimeUnit.MILLISECONDS.sleep(1000);
        
        return items;
    }
        
);

16:12:15.250 [pool-2-thread-5] INFO  AsyncJava - Getting items from the fridge
16:12:15.264 [pool-2-thread-1] INFO  AsyncJava - Getting items from the pantry
16:12:15.279 [pool-2-thread-2] INFO  AsyncJava - Getting items from the cupboard


#### Step 3 - Mixing the Items Together

The next process in the graph is taking all the items we collected from the fridge and the pantry, then mixing them together. To do this, we can utilize the **Futures.whenAllSucceed()** method to register an AsyncCallable function.

When both async running tasks complete, they will be combined in the AsyncCallable. This Callable will also run asynchronously. If we wish to run it synchronously, we use the **.call()** method as opposed to the **.callAsync()**;

```{java}
Futures.whenAllSucceed(
    ListenableFuture<T>, ...     // 1 or more ListenableFutures
)
.callAsync(
    AsyncCallabe<C>,      // AsyncCallable function
    Executor              // Executor
    )
    
 
returns ListenableFuture<T>
```


<img src="static/mixed.gif">

In [9]:
// Combining the fridgeItems & pantryItems ListenableFutures
ListenableFuture<String> mixedIngredients = Futures.whenAllSucceed(fridgeItems, pantryItems)
    .callAsync(
        () -> {
                for (String item : Futures.getDone(fridgeItems)){
                    logger.info("Mixing {}\n", item);
                    TimeUnit.MILLISECONDS.sleep(300);
                }
            
                for (String item : Futures.getDone(pantryItems)){
                    logger.info("Mixing {}\n", item);
                    TimeUnit.MILLISECONDS.sleep(300);
                }
        
               
               return Futures.immediateFuture("Batter");}
        , executor);

#### Step 4 - Pouring batter in the cupcake pan

After getting the Foil cups and the cupcake pan as well as mixing the batter, we are ready to pour the batter in each cup. This step requires the completion of the "mixing the ingredients" task as well as the "getting items from cupboard" task. 

<img src="static/readybake.gif">

In [10]:
// Combine cupboardItems and mixedIngredients ListenableFuture
ListenableFuture<String> readyToBake = Futures.whenAllSucceed(cupboardItems, mixedIngredients)
    .callAsync(
                () -> {
                    logger.info("Pouring batter into each cup in the pan\n");
                    TimeUnit.MILLISECONDS.sleep(5000);
                    return Futures.immediateFuture("Ready for Oven");
                }
                , executor);

#### Step 5 - Bake the cupcakes


<img src="static/baked.gif">

In [11]:
ListenableFuture<String> bakedCupcakes = Futures.whenAllSucceed(ovenPreheated, readyToBake)
    .call(
        () -> {
            boolean foo = Futures.getDone(ovenPreheated);
            logger.info("Baking the cupcakes\n");
            TimeUnit.MILLISECONDS.sleep(5000);
            return "Done";
        }
        , executor)

In [12]:
logger.info("{}", bakedCupcakes.get());

16:12:19.309 [pool-2-thread-3] INFO  AsyncJava - Mixing Eggs

16:12:19.611 [pool-2-thread-3] INFO  AsyncJava - Mixing Milk

16:12:19.926 [pool-2-thread-3] INFO  AsyncJava - Mixing Butter

16:12:20.228 [pool-2-thread-3] INFO  AsyncJava - Mixing Flour

16:12:20.543 [pool-2-thread-3] INFO  AsyncJava - Mixing Sugar

16:12:20.846 [pool-2-thread-3] INFO  AsyncJava - Mixing Baking Soda

16:12:21.160 [pool-2-thread-3] INFO  AsyncJava - Pouring batter into each cup in the pan
16:12:26.161 [pool-2-thread-3] INFO  AsyncJava - Baking the cupcakes
16:12:31.176 [IJava-executor-0] INFO  AsyncJava - Done
